# GPT Model w/ Descriptions
Combined_Songs_Artists.csv

Model trained on Playlist_Names, Playlist_Descriptions, Songs and Artists combined in one string

In [10]:
import pandas as pd
from datasets import Dataset
import ast
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import os
import csv
import torch

In [24]:
os.chdir('..')
os.getcwd()

str

In [11]:
#df2 = pd.read_csv('DS6051-Project/Playlist_data_with_lyrics.csv')
#df3 = pd.read_csv('DS6051-Project/playlist_data.csv')
df = pd.read_csv('../Additional Data/Combined_Songs_Artists.csv')

In [27]:
# Load and parse the dataset

# Convert stringified lists to actual lists
# for col in ['Playlist_Songs', 'Playlist_Artists']:
#     df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

df["Playlist_Song_IDs"] = df["Playlist_Song_IDs"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

def format_example(row):
    song_ids = [str(sid) for sid in row["Playlist_Song_IDs"] if sid is not None]
    playlist_body = "\n".join(song_ids)  # NEW target output
    return (
        f"### Prompt: {row['Playlist_Name']}\n"
        f"### Description: {row['Playlist_Description']}\n"
        f"### Playlist:\n{playlist_body}"
    )

df['text'] = df.apply(format_example, axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({"text": df['text'].tolist()})

In [28]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token

model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
model.resize_token_embeddings(len(tokenizer))  # In case we add special tokens

# Tokenize
def tokenize(batch):
    encodings = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    encodings["labels"] = encodings["input_ids"].copy()  # 🔥 Add this line
    return encodings
    
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map: 100%|██████████| 2289/2289 [00:15<00:00, 145.79 examples/s]


### Test Script

### Test Script

In [29]:
training_args = TrainingArguments(
    output_dir="./Models/gpt2_ID_Model_Checkpoints",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    fp16=True if torch.cuda.is_available() else False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss
50,5.225800
100,4.816000
150,4.744500
200,4.876500
250,4.734900
300,4.759800
350,4.614700
400,4.607400
450,4.665000
500,4.784200


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



TrainOutput(global_step=22900, training_loss=3.3895068825800867, metrics={'train_runtime': 3224.4019, 'train_samples_per_second': 14.198, 'train_steps_per_second': 7.102, 'total_flos': 4.251591779549184e+16, 'train_loss': 3.3895068825800867, 'epoch': 20.0})

In [30]:
model.save_pretrained("./Models/gpt2_ID_model")
tokenizer.save_pretrained("./Models/gpt2_ID_model")

('./Models/gpt2_ID_model/tokenizer_config.json',
 './Models/gpt2_ID_model/special_tokens_map.json',
 './Models/gpt2_ID_model/vocab.json',
 './Models/gpt2_ID_model/merges.txt',
 './Models/gpt2_ID_model/added_tokens.json')

In [31]:
def generate_playlist(prompt, max_length=200):
    input_text = f"### Prompt: {prompt}\n### Playlist:\n"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output = model.generate(
        input_ids = input_ids.to('cuda'),
        max_length=max_length,
        temperature=0.9,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result.split("### Playlist:\n")[1].strip()

# Try it out!
prompt = "Road Trip with Classic Rock"
print(generate_playlist(prompt))

5zVbV4c2Iy89JuYE6kUQk
0HPD5WQqrq7wPWR7P7Dw1i
5QTTqnJUZ67yVXexI5efjI
1yjY7rpaAQvKwpdUliHx0d
5A6OHHy73AR5tLxgTc98zz
1gihuPhrLraKYrJMAEONyc
0CAfXk7DXMnon4gLudAp7J
5rb9QrpfcKFHM1EUbSIurX
05qq0ZpZ6vbI1ky1Cg3NI
5Q0Nhxo0l2bP3pNjpGJwV1
